In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np # linear algebra
import pandas as pd #

Using TensorFlow backend.


In [2]:
topic_embedding= np.load( "esim_topic_embedding.npy" )
word_embedding= np.load( "esim_word_embedding.npy" )

qid_title=np.load("qtitlec.npy")
qid_1_title=np.load("qtitlec_1.npy")
qid_2_title=np.load("qtitlec_2.npy")
qid_3_title=np.load("qtitlec_3.npy")
qid_4_title=np.load("qtitlec_4.npy")
qid_5_title=np.load("qtitlec_5.npy")
qid_6_title=np.load("qtitlec_6.npy")

qid_topic=np.load("qtopic.npy")
qid_1_topic=np.load("qtopic_1.npy")
qid_2_topic=np.load("qtopic_2.npy")
qid_3_topic=np.load("qtopic_3.npy")
qid_4_topic=np.load("qtopic_4.npy")
qid_5_topic=np.load("qtopic_5.npy")
qid_6_topic=np.load("qtopic_6.npy")
l_topic=np.load("l_topic.npy")
u_topic=np.load("u_topic.npy")


In [3]:
feature=np.load("all_data_final.npy")

In [4]:
from keras.layers import *
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [5]:
def unchanged_shape(input_shape):
    "Function for Lambda layer"
    return input_shape
def compare(input_1, input_2):
    difft = Lambda(lambda x: K.abs(x[0] - x[1]), output_shape=unchanged_shape)([input_1, input_2])
    mult = Lambda(lambda x: x[0] * x[1], output_shape=unchanged_shape)([input_1, input_2])
    output=concatenate([difft, mult])
    return output


In [6]:
feature_shape=feature.shape[1]

In [8]:
def LSTUR(titlec_len=10,topic_len=10,word_embedding=word_embedding,topic_embedding_=topic_embedding):
    
    word_embedding = Embedding(
        input_dim=word_embedding.shape[0],
        output_dim=word_embedding.shape[1],
        weights=[word_embedding],
        input_length=titlec_len,
        trainable=False
    ) 
    
    topic_embedding = Embedding(
        input_dim=topic_embedding_.shape[0],
        output_dim=topic_embedding_.shape[1],
        weights=[topic_embedding_],
        input_length=topic_len,
        trainable=False
    ) 
    
    qid_title   = Input(shape=(titlec_len,), name="qid_title")
    qid_1_title = Input(shape=(titlec_len,), name="qid_1_title")
    qid_2_title = Input(shape=(titlec_len,), name="qid_2_title")
    qid_3_title = Input(shape=(titlec_len,), name="qid_3_title")
    qid_4_title = Input(shape=(titlec_len,), name="qid_4_title")
    qid_5_title = Input(shape=(titlec_len,), name="qid_5_title")
    qid_6_title = Input(shape=(titlec_len,), name="qid_6_title")
    
    qid_topic   = Input(shape=(topic_len,), name="qid_topic")
    qid_1_topic = Input(shape=(topic_len,), name="qid_1_topic")
    qid_2_topic = Input(shape=(topic_len,), name="qid_2_topic")
    qid_3_topic = Input(shape=(topic_len,), name="qid_3_topic")
    qid_4_topic = Input(shape=(topic_len,), name="qid_4_topic")
    qid_5_topic = Input(shape=(topic_len,), name="qid_5_topic")
    qid_6_topic = Input(shape=(topic_len,), name="qid_6_topic")
    
    l_topic = Input(shape=(50,), name="l_topic")
    
    ltopic_embedding = Embedding(
        input_dim=topic_embedding_.shape[0],
        output_dim=topic_embedding_.shape[1],
        weights=[topic_embedding_],
        input_length=50,
        trainable=False
    ) 
    l_topic_emb=ltopic_embedding(l_topic)
    
    conv   = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    conv_1 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    conv_2 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    conv_3 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    conv_4 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    conv_5 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    conv_6 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')
    
    qid_title_emb=word_embedding(qid_title)
    qid_1_title_emb=word_embedding(qid_1_title)
    qid_2_title_emb=word_embedding(qid_2_title)
    qid_3_title_emb=word_embedding(qid_3_title)
    qid_4_title_emb=word_embedding(qid_4_title)
    qid_5_title_emb=word_embedding(qid_5_title)
    qid_6_title_emb=word_embedding(qid_6_title)
    
    
    qid_topic_emb=topic_embedding(qid_topic)
    qid_1_topic_emb=topic_embedding(qid_1_topic)
    qid_2_topic_emb=topic_embedding(qid_2_topic)
    qid_3_topic_emb=topic_embedding(qid_3_topic)
    qid_4_topic_emb=topic_embedding(qid_4_topic)
    qid_5_topic_emb=topic_embedding(qid_5_topic)
    qid_6_topic_emb=topic_embedding(qid_6_topic)
    
    qid_title_rep=conv(qid_title_emb)
    qid_1_title_rep=conv_1(qid_1_title_emb)
    qid_2_title_rep=conv_2(qid_2_title_emb)
    qid_3_title_rep=conv_3(qid_3_title_emb)
    qid_4_title_rep=conv_4(qid_4_title_emb)
    qid_5_title_rep=conv_5(qid_5_title_emb)
    qid_6_title_rep=conv_6(qid_6_title_emb)
    
    qid_title_rep   = Attention(titlec_len)(qid_title_rep)
    qid_1_title_rep = Attention(titlec_len)(qid_1_title_rep)
    qid_2_title_rep = Attention(titlec_len)(qid_2_title_rep)
    qid_3_title_rep = Attention(titlec_len)(qid_3_title_rep)
    qid_4_title_rep = Attention(titlec_len)(qid_4_title_rep)
    qid_5_title_rep = Attention(titlec_len)(qid_5_title_rep)
    qid_6_title_rep = Attention(titlec_len)(qid_6_title_rep)
 #   return 0
    qtitle_history =concatenate([qid_1_title_rep,qid_2_title_rep,qid_3_title_rep,qid_4_title_rep,qid_5_title_rep,qid_6_title_rep])
    qtitle_history=Reshape((6,64))(qtitle_history)
    GRU_Encode1=Bidirectional(CuDNNGRU(128, return_sequences=True))
    GRU_Encode2=Bidirectional(CuDNNGRU(32))
    qtitle_history=GRU_Encode1(qtitle_history)
    qtitle_history=GRU_Encode2(qtitle_history)
    qtopic_history=concatenate([qid_1_topic_emb,qid_2_topic_emb,qid_3_topic_emb,qid_4_topic_emb,qid_5_topic_emb,qid_6_topic_emb],axis=1)
    
    u_rep=concatenate([qtitle_history,GlobalAveragePooling1D()(qtopic_history),GlobalAveragePooling1D()(l_topic_emb)])
    
    q_rep=concatenate([qid_title_rep,GlobalAveragePooling1D()(qid_topic_emb),GlobalAveragePooling1D()(qid_topic_emb)])
    feature = Input(shape=(feature_shape,), name="feature")
    feature_fc= Dense(1024, activation='relu', )(feature)
    feature_fc = Dropout(0.3)(feature_fc)
    feature_fc = Dense(512, activation='relu', )(feature_fc)
    feature_fc = Dropout(0.3)(feature_fc)
    out2=compare(u_rep, q_rep)
    out=concatenate([out2,feature_fc])
    out = Dense(1024,activation='relu')(out)
    feature_fc = Dropout(0.3)(feature_fc)
    out = Dense(512,activation='relu')(out)
    output = Dense(1,activation='sigmoid')(out)
    
    model = Model(inputs=[qid_title,qid_1_title,qid_2_title,qid_3_title,qid_4_title,qid_5_title,qid_6_title,
                          qid_topic,qid_1_topic,qid_2_topic,qid_3_topic,qid_4_topic,qid_5_topic,qid_6_topic,
                          l_topic,feature], outputs=output)

    model.compile(optimizer ='adam',loss= 'binary_crossentropy',metrics=['acc'])
    #model.summary()
    return model
#LSTUR()

In [6]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
target=train['target']

In [7]:
sub1 = np.zeros((test.shape[0],1 ))
oof_pref1 = np.zeros((train.shape[0], 1))
count=0

In [8]:
def make_data(index_):
    data=dict()
    data['qid_title']=qid_title[index_]
    data['qid_1_title']=qid_1_title[index_]
    data['qid_2_title']=qid_2_title[index_]
    data['qid_3_title']=qid_3_title[index_]
    data['qid_4_title']=qid_4_title[index_]
    data['qid_5_title']=qid_5_title[index_]
    data['qid_6_title']=qid_6_title[index_]
    data['qid_topic']=qid_topic[index_]
    data['qid_1_topic']=qid_1_topic[index_]
    data['qid_2_topic']=qid_2_topic[index_]
    data['qid_3_topic']=qid_3_topic[index_]
    data['qid_4_topic']=qid_4_topic[index_]
    data['qid_5_topic']=qid_5_topic[index_]
    data['qid_6_topic']=qid_6_topic[index_]
    data['l_topic']=l_topic[index_]
    return data

In [9]:
test_data=make_data(range(len(train),len(train)+len(test)))

In [12]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics

skf = StratifiedKFold(n_splits=5, random_state=1996, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(train, target)):
    K.clear_session()
    filepath = "lstur_nofea_model%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', factor=0.8, patience=1, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_loss', min_delta=0.0001, patience=2,verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    print(index)
    nnmodel = LSTUR()

    train_y, test_y =target[train_index], target[test_index]
    train_x=make_data(train_index)
    test_x=make_data(test_index)

    history = nnmodel.fit(train_x, train_y, batch_size=2048, epochs=10, verbose=1, validation_data=(test_x, test_y),callbacks=callbacks,)
    nnmodel.load_weights(filepath)
    

    sub1 += nnmodel.predict(test_data, batch_size=2048)
    oof_pref1[test_index] = nnmodel.predict(test_x, batch_size=2048)
    count += 1
    print('val_auc:',metrics.roc_auc_score(test_y,oof_pref1[test_index]))

0
Train on 7591329 samples, validate on 1897833 samples
Epoch 1/10
7591329/7591329 [==============================] - 191s 25us/step - loss: 0.4425 - acc: 0.8227 - val_loss: 0.4392 - val_acc: 0.8229

Epoch 00001: val_loss improved from inf to 0.43923, saving model to lstur_nofea_model0.h5
Epoch 2/10
7591329/7591329 [==============================] - 182s 24us/step - loss: 0.4357 - acc: 0.8230 - val_loss: 0.4356 - val_acc: 0.8230

Epoch 00002: val_loss improved from 0.43923 to 0.43562, saving model to lstur_nofea_model0.h5
Epoch 3/10
7591329/7591329 [==============================] - 178s 23us/step - loss: 0.4323 - acc: 0.8232 - val_loss: 0.4341 - val_acc: 0.8230

Epoch 00003: val_loss improved from 0.43562 to 0.43412, saving model to lstur_nofea_model0.h5
Epoch 4/10
7591329/7591329 [==============================] - 175s 23us/step - loss: 0.4291 - acc: 0.8235 - val_loss: 0.4326 - val_acc: 0.8232

Epoch 00004: val_loss improved from 0.43412 to 0.43256, saving model to lstur_nofea_model0

In [35]:
sub1/=5
stacking=pd.concat([pd.DataFrame(oof_pref1),pd.DataFrame(sub1)])
stacking.columns=['nn_LSTUR']
stacking=stacking.reset_index(drop=True)
stacking.to_hdf('stacking_nn_LSTUR_cv68.h5', key='data')

In [13]:
sub2 = np.zeros((test.shape[0],1 ))
oof_pref2 = np.zeros((train.shape[0], 1))
count=0

In [15]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics

skf = StratifiedKFold(n_splits=5, random_state=1996, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(train, target)):
    K.clear_session()
    filepath = "lstur_nofea_model%d.h5" % index
    model = LSTUR()
    print(index)
    train_x=make_data(train_index)
    test_x=make_data(test_index)
    train_y, test_y =target[train_index], target[test_index]
    print('start')
    model.load_weights(filepath)
    sub2 += model.predict(make_data(range(len(train),len(train)+len(test))), batch_size=4000)
    oof_pref2[test_index] = model.predict(test_x, batch_size=4000)
    count += 1
    print('val_auc:',metrics.roc_auc_score(test_y,oof_pref2[test_index]))

0
start
val_auc: 0.6959959662500637
1
start
val_auc: 0.6961347025202749
2
start
val_auc: 0.6959837786736491
3
start
val_auc: 0.6966664228985495
4
start
val_auc: 0.6948296389189167


In [19]:
stacking=pd.concat([pd.DataFrame(oof_pref2),pd.DataFrame(sub2/5)])
stacking.columns=['lstur_no_feature']
stacking=stacking.reset_index(drop=True)
stacking.to_hdf('stackinglstur_wnf.h5', key='data')